### 1사분면 HAHV (Excitment) : arousal > 3, valence > 3   
### 2사분면 HALV (Stress) : arousal > 3, valence < 3   
### 3사분면 LALV (해당사항 없음)   
### 4사분면 LAHV (Nertual) : arousal < 3, valence > 3 (연구에서 사용되지 않음)


In [ ]:
import pickle
import numpy as np
import data_utils as du
import scipy.io as sciio
import matplotlib.pyplot as plt

from dataclasses import dataclass
from typing import List
from sklearn.preprocessing import MinMaxScaler



@dataclass
class PersonData:
    ecg_baseline: List
    ecg_stimuli: List
    valance: List
    arousal: List
        
@dataclass
class FilmData:
    ecg_baseline: np.ndarray
    ecg_stimuli: np.ndarray
    valance: float
    arousal: float

def iterate_persons(ppl_array):
    for person_idx in range(len(ppl_array)):
        #print('person:', person_idx)
        ppl_struct = ppl_array[person_idx]
        ecg_struct = ppl_struct['ECG'][0][0]
        films_baseline_array = ecg_struct['baseline'][0][0]
        films_stimuli_array = ecg_struct['stimuli'][0][0]
        #print(ppl_struct.dtype)
        valance = ppl_struct['ScoreValence'][0][0]
        arousal = ppl_struct['ScoreArousal'][0][0]
        assert len(films_baseline_array) == len(
            films_stimuli_array), "len of baseline and simuli array not the same. Not every film is recorded fully maybe?"
        data = PersonData(films_baseline_array, films_stimuli_array, valance, arousal)
        yield data
        
        
def iterate_films(p_data: PersonData):
    for film_idx in range(len(p_data.ecg_baseline)):
        #print('film_idx : ', film_idx)
        d_ecg_baseline = p_data.ecg_baseline[film_idx][0]
        d_ecg_stimuli = p_data.ecg_stimuli[film_idx][0]
        valance = p_data.valance[film_idx][0]
        arousal = p_data.arousal[film_idx][0]
        data = FilmData(d_ecg_baseline, d_ecg_stimuli, valance, arousal)
        yield data
        
def data_minmax(data):
    
    scaler = MinMaxScaler()

    try:
        scaled_data = scaler.fit_transform(data)
        
    except ValueError:
        scaled_data = scaler.fit_transform(data.reshape(-1,1))
    
    return scaled_data

In [ ]:
# .mat 파일 불러오기 
mat_path = "D:\Database\Dreamer\DREAMER.mat"
data_path = mat_path.replace("\\", "/")
save_path = 'D:/Dataset/Denoised.pkl'

data = sciio.loadmat(data_path)
#데이터 불러오기 생략 

ppl_array = data['DREAMER'][0][0]['Data'][0] # 23명의 사람이 담긴 데이터
print(ppl_array.shape)


Fs = 256
high_cut = 0.5
low_cut = 100
low, high = 57 ,63
order = 2

neutral_data, st_data = [], []
excitement_data, stress_data, base_data = [], [], []


for p_data in iterate_persons(ppl_array):
    for f_data in iterate_films(p_data):
        
        
        base = f_data.ecg_baseline[:,0]
        st_data = f_data.ecg_stimuli[:,0]
        
        # Signal Denoised
        base = du.butter_highpass(base.flatten(), high_cut, Fs, order)
        st_data = du.butter_highpass(st_data.flatten(), high_cut, Fs, order)

        base = du.butter_bandstop(base, low, high, Fs, order)
        st_data = du.butter_bandstop(st_data, low, high, Fs, order)

        base = du.butter_lowpass(base, low_cut, Fs, order)
        st_data = du.butter_lowpass(st_data, low_cut, Fs, order)
        
        
        # Signal Normalization
        base = data_minmax(base)
        st_data = data_minmax(st_data)
        
        
        # Neutral Class
        neutral_data.append(base)
        
        
        # 1 사분면 - Excitement
        if f_data.arousal > 3 and f_data.valance > 3:
            excitement_data.append(st_data)
            
        # 2 사분면 - Stress
        elif f_data.arousal > 3 and f_data.valance < 3:
            stress_data.append(st_data)
            
        # 3 사분면 - Other
        elif f_data.arousal < 3 and f_data.valance > 3:
            base_data.append(st_data)
            
        # 4 사분면 - Other
        else:
            pass

In [ ]:
# Data
excitement = np.concatenate(np.array(excitement_data)).flatten()
neutral = np.array(neutral_data).flatten()
stress = np.concatenate(np.array(stress_data)).flatten()

# Crate Data dict 
data_dict = {'baseline' : neutral.flatten(), 'excitement' : excitement.flatten(), 'stress' : stress.flatten()}

# Save dataset 
with open(save_path, 'wb') as f:
    pickle.dump(data_dict, f, protocol=pickle.HIGHEST_PROTOCOL)